In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.models import Model
import warnings
from tensorflow.keras.layers import Dense, Flatten, Activation, Dropout, Embedding, Conv1D, Conv2D, MaxPooling2D, \
    MaxPooling1D, Concatenate, BatchNormalization, GaussianNoise
from tensorflow.keras.layers import LSTM, TimeDistributed, Permute, Reshape, Lambda, RepeatVector, Input, Multiply, \
    SimpleRNN, GRU, LeakyReLU
import site
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical
import h5py as h5
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import errno
import os
from collections import defaultdict
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.summary import create_file_writer
from sklearn.model_selection import train_test_split


In [7]:
%pip install keras-tuner

  Using cached https://files.pythonhosted.org/packages/20/ec/1ef246787174b1e2bb591c95f29d3c1310070cad877824f907faba3dade9/keras-tuner-1.0.2.tar.gz
  Using cached https://files.pythonhosted.org/packages/9b/c4/4a21174f32f8a7e1104798c445dacdc1d4df86f2f26722767034e4de4bff/terminaltables-3.1.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/44/98/5b86278fbbf250d239ae0ecb724f8572af1c91f4a11edf4d36a206189440/colorama-0.4.4-py2.py3-none-any.whl
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=0bed48ef131605f52cf99c5d49d152241c93535d350c3f8a1d5b7d7e77a5d9dc
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=008bc0ed858241d2177f88721171670ab953fb87c530f8d562cb8fd4f0b3c1fd
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Suc

In [8]:
%pip install keras_self_attention

  Using cached https://files.pythonhosted.org/packages/c3/34/e21dc6adcdab2be03781bde78c6c5d2b2136d35a1dd3e692d7e160ba062a/keras-self-attention-0.49.0.tar.gz
  Created wheel for keras-self-attention: filename=keras_self_attention-0.49.0-cp37-none-any.whl size=19468 sha256=7a53e06eeccc429c65b60d630a8bccc1d959d9375a805144c9c36fd9d541872d
  Stored in directory: /root/.cache/pip/wheels/6f/9d/c5/26693a5092d9313daeae94db04818fc0a2b7a48ea381989f34
Successfully built keras-self-attention


In [9]:
from keras_self_attention import SeqSelfAttention, SeqWeightedAttention
import kerastuner as kt


In [10]:
site.addsitedir("D:/Code/ncRNA")
pd.set_option('display.width', 400)
pd.set_option('display.max_columns', 40)


In [ ]:
def getData(is500=True, shuffle=False, ise2e=False, include_secondary=False, validation_split=None):
    if not include_secondary:
        hf_Train = h5.File(
            f'./data/{"e2e_Train_Data" if ise2e else "Fold_10_Train_Data"}_{str(500) if is500 else str(1000)}.h5', 'r')
        hf_Test = h5.File(
            f'./data/{"e2e_Test_Data" if ise2e else "Fold_10_Test_Data"}_{str(500) if is500 else str(1000)}.h5', 'r')
    else:
        hf_Train = h5.File(f'./data/e2e_Train_Secondary_Data_1136.h5', 'r')
        hf_Test = h5.File(f'./data/e2e_Test_Secondary_Data_1136.h5', 'r')

    X_train = hf_Train['Train_Data']  # Get train set
    X_train = np.array(X_train)
    Y_train = hf_Train['Label']  # Get train label
    Y_train = np.array(Y_train)
    X_test = hf_Test['Train_Data']  # Get test set
    X_test = np.array(X_test)
    Y_test = hf_Test['Label']  # Get test label
    Y_test = np.array(Y_test)
    Y_train = to_categorical(Y_train, 13)  # Process the label of tain
    Y_test = to_categorical(Y_test, 13)  # Process the label of te

    if shuffle:
        X_train, Y_train = coShuffled_vectors(X_train, Y_train)
        X_test, Y_test = coShuffled_vectors(X_test, Y_test)

    X_validation = Y_validation = None
    if validation_split is not None:
        # sklearn split shuffles anyway
        X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=validation_split)

    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation